In [2]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload
import sys
from __future__ import division
sys.path.append(r"C:\dev\opinion\opinion\python\opinion")
import os
import matplotlib.pyplot as plt

import utils.fileutils as fileutils
import utils.nlputils as nlputils
import utils.metrics as metrics

import cPickle as pickle
import pandas as pd
import numpy as np
import scipy as sp

from scipy.stats import kurtosis
from scipy.stats import skew
from scipy.stats import mannwhitneyu
from scipy.stats import spearmanr
from scipy.stats import kendalltau
from scipy.stats import chisquare
from scipy.stats import chi2_contingency
from scipy.stats import pearsonr
from math import sqrt

import datetime
from math import sqrt

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
indir = r"C:\dev\opinion\papers\covid19\CovidDevRantOpenCoding"
infileXL = os.path.join(indir, "CovidTagContentsWithCategoryMajorRevisionWithFinalCategories.xlsx")

In [5]:
df = pd.read_excel(infileXL)
df.head(3)

,id,creation_time,score,num_comments,tags,has_image,text,Relevant,Label1,Label2,...,Emotion2,Emotion3,sarcasm,irony,Category1,Category2,Category3,Category4,FinalCategory4,FinalCategory3
0,2438510,2020-03-21 15:17:25,106,17,rant,NaN,Clickbait will never change. \nIt's the most s...,N,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2423716,2020-03-05 07:31:51,95,10,"joke/meme,covid-19,corona,bug,ticket",NaN,COVID-19\nWhy it sounds like a JIRA ticket?,Y,joke/meme,bug,...,NaN,NaN,Y,NaN,NaN,"joke/meme,coding & maintenance","entertainment,development","Wellbeing,Technical",Personal & Family Wellbeing|Technical Aspects,Development|Entertainment
2,2452432,2020-04-05 18:52:13,66,7,"devrant,corona virus",NaN,"*Corona Virus Lock-down*\n-University Prof: ""N...",Y,lockdown,work life balance,...,NaN,NaN,Y,NaN,NaN,"quarantine,Freetime/Vacation","pandemic,work/life work/life balance","Emergency ,Wellbeing",Lockdown Preparedness|Personal & Family Wellbeing,Coping|Work Life Balance


# compute co-occurrence of topic categories

In [6]:
fcats = ['lockdown preparedness', 'technical aspects', 'personal & family wellbeing', 
       'worplace & professional aspects', 'financial concerns', 'education & societal concerns']

In [9]:
isnullFinalCategory4 = pd.isnull(df["FinalCategory4"])

colocates = dict()

for i, c1 in enumerate(fcats):
    #colocates[c1] = dict()
    for j, c2 in enumerate(fcats):
        if i < j : continue
        if c1 == c2: continue
        key = c1+'=>'+c2
        colocates[key] = 0

for index, row in df.iterrows():
    if isnullFinalCategory4[index]: continue
    isrelevant = row["Relevant"]
    if isrelevant!= 'Y': continue
    cat4 = row["FinalCategory4"]
    cats = [c.lower().strip() for c in cat4.split("|")]
    for l1 in cats:
        for l2 in cats:
            if l1 == l2: continue
            key1 =  l1+'=>'+l2
            key2 =  l2+'=>'+l1
            if key1 in colocates:
                colocates[key1] += 1
            elif key2 in colocates: 
                colocates[key2] += 1
            else: 
                print "Absent = ", key1, '|', key2
for c in colocates:
    print c, colocates[c]

education & societal concerns=>personal & family wellbeing 46
education & societal concerns=>financial concerns 28
financial concerns=>technical aspects 38
worplace & professional aspects=>lockdown preparedness 148
financial concerns=>lockdown preparedness 46
personal & family wellbeing=>lockdown preparedness 238
education & societal concerns=>worplace & professional aspects 94
personal & family wellbeing=>technical aspects 184
education & societal concerns=>lockdown preparedness 88
technical aspects=>lockdown preparedness 126
worplace & professional aspects=>personal & family wellbeing 230
worplace & professional aspects=>technical aspects 188
financial concerns=>personal & family wellbeing 46
education & societal concerns=>technical aspects 38
financial concerns=>worplace & professional aspects 108


In [10]:
# compute MW test of occurrence distribtution

def computeDistributionBetween(c1, c2):
    dist = {c1: [], c2: []}
    isnullFinalCategory4 = pd.isnull(df["FinalCategory4"])
    for index, row in df.iterrows():
        if isnullFinalCategory4[index]: continue
        isrelevant = row["Relevant"]
        if isrelevant!= 'Y': continue
        cat4 = row["FinalCategory4"]
        cats = [c.lower().strip() for c in cat4.split("|")]
        
        if c1 in cats:
            dist[c1].append(1)
        else:
            dist[c1].append(0)
        if c2 in cats:
            dist[c2].append(1)
        else:
            dist[c2].append(0)
    return dist

def cohen_d(x,y):
    nx = len(x)
    ny = len(y)
    dof = nx + ny - 2
    return (np.mean(x) - np.mean(y)) / sqrt(((nx-1)*np.std(x, ddof=1) ** 2 + (ny-1)*np.std(y, ddof=1) ** 2) / dof)

def computeSignificance(c1, c2):
    dist = computeDistributionBetween(c1, c2)
    l1 = dist[c1]
    l2 = dist[c2]
    m1 = []
    m2 = []
    for i, k1 in enumerate(l1):
        k2 = l2[i]
        if k1 == 0 and k2 == 0: continue
        m1.append(k1)
        m2.append(k2)
        
    #print "MW"
    s, p = mannwhitneyu(m1, m2)
    print s, p, p <= 0.05
    #print "Cohen"
    #print cohen_d(dist[c1], dist[c2])
def computeCorrelation(c1, c2):
    dist = computeDistributionBetween(c1, c2)
    l1 = dist[c1]
    l2 = dist[c2]
    m1 = []
    m2 = []
    for i, k1 in enumerate(l1):
        k2 = l2[i]
        if k1 == 0 and k2 == 0: continue
        m1.append(k1)
        m2.append(k2)    
    s, p = kendalltau(m1, m2)
    print s, p, p<=0.05
def computeIndependence(c1, c2):
        dist = computeDistributionBetween(c1, c2)
        l1 = dist[c1]
        l2 = dist[c2]
        m1 = []
        m2 = []
        for i, k1 in enumerate(l1):
            k2 = l2[i]
            if k1 == k2 and k1 == 0: continue
            #if k1 == 0 and k2 == 0: continue
            m1.append(k1)
            m2.append(k2)   
        print len(m1)
        obs = np.array([m1,m2])
        #g, p, dof, expctd = chi2_contingency(np.array([dist[c1], dist[c2]]), lambda_="log-likelihood")
        g, p, dof, expctd = chi2_contingency(obs, lambda_="log-likelihood")
        msg = 'Has Relationship'
        if p <= 0.05: 
            msg = 'No Relationship'
        print g, p, dof, p<=0.05, msg
        #print s, p, p<=0.05
def pearson(c1, c2):
        dist = computeDistributionBetween(c1, c2)
        l1 = dist[c1]
        l2 = dist[c2]
        m1 = []
        m2 = []
        for i, k1 in enumerate(l1):
            k2 = l2[i]
            if k1 == 0 and k2 == 0: continue
            m1.append(k1)
            m2.append(k2)   
        s, p = pearsonr(m1, m2)
        #print r[0,1]
        #obs = np.array([m1,m2])
        #g, p, dof, expctd = chi2_contingency(np.array([dist[c1], dist[c2]]), lambda_="log-likelihood")
        #g, p, dof, expctd = chi2_contingency(obs, lambda_="log-likelihood")
        #print g, p, dof, p<=0.05
        print s, p, p<=0.05
    

In [11]:
for i, c1 in enumerate(fcats):
    for j, c2 in enumerate(fcats):
        if c1 == c2: continue
        if i > j: continue
        print c1,'<=>' , c2
        #computeSignificance(c1, c2)
        #computeCorrelation(c1, c2)
        computeIndependence(c1, c2)
        #pearson(c1, c2)
        print "========================="

lockdown preparedness <=> technical aspects
486
585.8760189384219 0.0011093684940903406 485 True No Relationship
lockdown preparedness <=> personal & family wellbeing
494
516.2532575012416 0.2265211453700183 493 False Has Relationship
lockdown preparedness <=> worplace & professional aspects
634
747.6505335727365 0.0010903294331085593 633 True No Relationship
lockdown preparedness <=> financial concerns
380
427.052365983748 0.044512243376406364 379 True No Relationship
lockdown preparedness <=> education & societal concerns
352
351.5735604399367 0.4813372727706143 351 False Has Relationship
technical aspects <=> personal & family wellbeing
504
564.2675244800259 0.030109250088757696 503 True No Relationship
technical aspects <=> worplace & professional aspects
597
660.3900666987622 0.03430901115057688 596 True No Relationship
technical aspects <=> financial concerns
367
425.8092089647387 0.016799997523696892 366 True No Relationship
technical aspects <=> education & societal concerns
36

In [91]:
from scipy.stats import chi2_contingency
obs = np.array([[10, 10, 20], [20, 20, 20]])
chi2_contingency(obs)

(2.7777777777777777, 0.24935220877729622, 2L, array([[12., 12., 16.],
        [18., 18., 24.]]))

In [92]:
obs

array([[10, 10, 20],
       [20, 20, 20]])

In [116]:
from scipy.stats import chi2_contingency
c1 = fcats[1]
c2 = fcats[2]
dist = computeDistributionBetween(c1, c2)
l1 = dist[c1]
l2 = dist[c2]
ll1 = []
ll2 = []
for i, l in enumerate(l1):
    k = l2[i]
    if l == k and l ==0: continue
    ll1.append(l)
    ll2.append(k)

obs = np.array([ll1,ll2])
chi2_contingency(obs, lambda_="log-likelihood")

(564.2675244800259,
 0.030109250088757696,
 503L,
 array([[0.89261745, 0.44630872, 0.89261745, ..., 0.44630872, 0.44630872,
         0.44630872],
        [1.10738255, 0.55369128, 1.10738255, ..., 0.55369128, 0.55369128,
         0.55369128]]))

In [115]:
obs

array([[1, 0, 0, ..., 0, 0, 0],
       [1, 1, 0, ..., 0, 0, 0]])

# compute distribution of topic for a given topic category for the cooccurrence of two topic categories

In [125]:
# 1. find all posts where c1 and c2 co-occur
# 2. find all topics under c1 or c2 that are found in those common posts
# 3. count the occurrence of the topics among the common posts 

In [12]:
def findCollocatedPosts():
    isnullFinalCategory4 = pd.isnull(df["FinalCategory4"])
    colocates = dict()
    for i, c1 in enumerate(fcats):
        #colocates[c1] = dict()
        for j, c2 in enumerate(fcats):
            if i < j : continue
            if c1 == c2: continue
            key = c1+'=>'+c2
            colocates[key] = []
    for index, row in df.iterrows():
        if isnullFinalCategory4[index]: continue
        isrelevant = row["Relevant"]
        if isrelevant!= 'Y': continue
        cat4 = row["FinalCategory4"]
        cats = [c.lower().strip() for c in cat4.split("|")]
        for l1 in cats:
            for l2 in cats:
                if l1 == l2: continue
                key1 =  l1+'=>'+l2
                key2 =  l2+'=>'+l1
                if key1 in colocates:
                    colocates[key1].append(row)
                elif key2 in colocates: 
                    colocates[key2].append(row)
    return colocates

In [13]:
#c1 = fcats[0]
#c1 = fcats[1]
colocates = findCollocatedPosts()

In [132]:
colocates.keys()

['education & societal concerns=>personal & family wellbeing',
 'education & societal concerns=>financial concerns',
 'financial concerns=>technical aspects',
 'worplace & professional aspects=>lockdown preparedness',
 'financial concerns=>lockdown preparedness',
 'personal & family wellbeing=>lockdown preparedness',
 'education & societal concerns=>worplace & professional aspects',
 'personal & family wellbeing=>technical aspects',
 'education & societal concerns=>lockdown preparedness',
 'technical aspects=>lockdown preparedness',
 'worplace & professional aspects=>personal & family wellbeing',
 'worplace & professional aspects=>technical aspects',
 'financial concerns=>personal & family wellbeing',
 'education & societal concerns=>technical aspects',
 'financial concerns=>worplace & professional aspects']

In [133]:
colocates['education & societal concerns=>personal & family wellbeing'][0]

id                                                          2437976
creation_time                                   2020-03-20 21:40:10
score                                                            31
num_comments                                                     24
tags                                  rant,difficult family,fuck it
has_image                                                       NaN
text              !dev\n\nDay 8 in isolation.\nIm lonely..\nMy b...
Relevant                                                          Y
Label1                                                    isolation
Label2                                                   loneliness
Label3                                               family/friends
Label4                                                     politics
Label5                                                          NaN
Label6                                                          NaN
polarity                                        

In [14]:
len(colocates['education & societal concerns=>personal & family wellbeing'])

46

In [28]:
mapTopic = {
    'awareness'      : 'Awareness',
    'business'       : 'Business',
    'development'    : 'Development',
    'domestic'       : 'Work Life Balance',
    'education'      : 'Education',
    'entertainment'  : 'Entertainment',
    'environment'    : 'Environment',
    'ergonomics'     : 'Ergonomics',
    'health'         : 'Health & Safety',
    'job'            : 'Jobs & Finance',
    'pandemic'       : 'Coping',
    'politics'       : 'Law & Politics',
    'productivity'   : 'Productivity',
    'relationship'   : 'Relationship',
    'security'       : 'Security',
    'social'         : 'Family & Friends',
    'technology'     : 'Technology',
    'telework'       : 'Telework',
    'travel'         : 'Travel Issues',
    'work/life work/life balance': 'Work Life Balance'
}
mapTopicCategory = {
    'financial': 'Financial Concerns',
    'emergency': 'Lockdown Preparedness',
    'technical': 'Technical Aspects',
    'wellbeing': 'Personal & Family Wellbeing',
    'societal' : 'Education & Societal Concerns',
    'workplace': 'Worplace & Professional Aspects'
}
topicByCategories = {
    'financial': ['job', 'business'],
    'emergency': ['pandemic', 'awareness'],
    'technical': ['development', 'technology', 'security'],
    'wellbeing': ['entertainment', 'health', 'social', 'work/life work/life balance'],
    'societal' : ['education', 'politics', 'environment'],
    'workplace': ['telework', 'relationship', 'productivity', 'ergonomics', 'travel']
}
topicByCategoriesMapped = {
    'financial': ['Jobs & Finance', 'Business'],
    'emergency': ['Coping', 'Awareness'],
    'technical': ['Development', 'Technology', 'Security'],
    'wellbeing': ['Entertainment', 'Health & Safety', 'Family & Friends', 'Work Life Balance'],
    'societal' : ['Education', 'Law & Politics', 'Environment'],
    'workplace': ['Telework', 'Relationship', 'Productivity', 'Ergonomics', 'Travel Issues']
}



In [29]:
def findTopicForTwoCategories(c1,c2,c3):
    mc1 = mapTopicCategory[c1].lower()
    mc2 = mapTopicCategory[c2].lower()
    mc3 = mapTopicCategory[c3].lower()
    key1 = mc1+'=>'+mc2
    key2 = mc2+'=>'+mc1
    if key1 in colocates:
        records = colocates[key1]
    elif key2 in colocates:
        records = colocates[key2]
    else:
        print "Do not exist keys: ", key1, ' and ', key2
        return
    topics = dict()
    tot = len(records)
    for record in records:
        cat3 = record["FinalCategory3"]
        #cat3 = cat3.lower()
        cats = [cat.strip() for cat in cat3.split("|")]
        for cat in cats:
            if cat in topicByCategoriesMapped[c3]:
                if cat not in topics: 
                    topics[cat] = 0
                topics[cat] += 1
    fracs = dict()
    for cat in topics:
        fracs[cat] = topics[cat]*100/tot
    print topics
    print fracs
    print tot

In [30]:
c1 = 'workplace'
c2 = 'wellbeing'
c3 = c1
for c2 in topicByCategories:
    print c1, c2
    findTopicForTwoCategories(c1,c2,c3)
    print "."*80

workplace financial
{u'Travel Issues': 8, u'Productivity': 8, u'Relationship': 54, u'Telework': 56}
{u'Travel Issues': 7.407407407407407, u'Productivity': 7.407407407407407, u'Relationship': 50.0, u'Telework': 51.851851851851855}
108
................................................................................
workplace emergency
{u'Travel Issues': 12, u'Productivity': 8, u'Ergonomics': 6, u'Relationship': 50, u'Telework': 112}
{u'Travel Issues': 8.108108108108109, u'Productivity': 5.405405405405405, u'Ergonomics': 4.054054054054054, u'Relationship': 33.78378378378378, u'Telework': 75.67567567567568}
148
................................................................................
workplace technical
{u'Travel Issues': 8, u'Productivity': 26, u'Ergonomics': 12, u'Relationship': 42, u'Telework': 152}
{u'Travel Issues': 4.25531914893617, u'Productivity': 13.829787234042554, u'Ergonomics': 6.382978723404255, u'Relationship': 22.340425531914892, u'Telework': 80.85106382978724}
188
..

In [57]:
c1 = 'societal'
c3 = c1
for c2 in topicByCategories:
    if c1 == c2: continue
    print c1, c2
    findTopicForTwoCategories(c1,c2,c3)
    print "."*80

societal financial
{u'Law & Politics': 20, u'Education': 8}
{u'Law & Politics': 71.42857142857143, u'Education': 28.571428571428573}
28
................................................................................
societal emergency
{u'Environment': 10, u'Law & Politics': 44, u'Education': 38}
{u'Environment': 11.363636363636363, u'Law & Politics': 50.0, u'Education': 43.18181818181818}
88
................................................................................
societal technical
{u'Law & Politics': 16, u'Education': 22}
{u'Law & Politics': 42.10526315789474, u'Education': 57.89473684210526}
38
................................................................................
societal wellbeing
{u'Environment': 2, u'Law & Politics': 34, u'Education': 10}
{u'Environment': 4.3478260869565215, u'Law & Politics': 73.91304347826087, u'Education': 21.73913043478261}
46
................................................................................
societal workplace
{u'Environment'

In [144]:
c1 = 'workplace'
c2 = 'wellbeing'
c3 = c2
findTopicForTwoCategories(c1,c2,c3)

{u'entertainment': 58}
{u'entertainment': 25.217391304347824}
230


In [41]:
def getPostInfoForTopicForTwoCategories(c1, c2, c3):
    mc1 = mapTopicCategory[c1].lower()
    mc2 = mapTopicCategory[c2].lower()
    mc3 = mapTopicCategory[c3].lower()
    key1 = mc1+'=>'+mc2
    key2 = mc2+'=>'+mc1
    if key1 in colocates:
        records = colocates[key1]
    elif key2 in colocates:
        records = colocates[key2]
    topics = dict()
    for record in records:
        cat3 = record["FinalCategory3"]
        #cat3 = cat3.lower()
        cats = [cat.strip() for cat in cat3.split("|")]
        for cat in cats:
            if cat in topicByCategoriesMapped[c3]:
                if cat not in topics: 
                    topics[cat] = []
                topics[cat].append({'id': record["id"], "text": record["text"]})
    return topics

In [147]:
c1 = 'workplace'
c2 = 'wellbeing'
c3 = c1
topics = getPostInfoForTopicForTwoCategories(c1,c2,c3)
topics['telework'][0:5]

[{'id': 2439836L,
  'text': u"#TheCoronaEffect\n\nBefore Corona: (Work From Office)\n\nBoss: Let's have a call.\nMe: Sure, allow me some time I am assisting the team on a new feature in the app.\nBoss: Ok, ping me as you get free.\n\n----------------------------------------------------------\n\nNow: (Work From Home)\n\nBoss: ***Calls for the 15th time in a day...***\nMe: (With Bleeding Ears) Yes sir, am here...!\n(Having to pick up every single time as he knows you've got nowhere to go 'coz the whole city is in LockDown)\nBoss: ***Talks for another 1 hour with screen share***\n\nMy Boss is a bigger threat to my health than Corona now!!!\n\n#GoCoronaGo"},
 {'id': 2439836L,
  'text': u"#TheCoronaEffect\n\nBefore Corona: (Work From Office)\n\nBoss: Let's have a call.\nMe: Sure, allow me some time I am assisting the team on a new feature in the app.\nBoss: Ok, ping me as you get free.\n\n----------------------------------------------------------\n\nNow: (Work From Home)\n\nBoss: ***Calls f

In [42]:
c1 = 'wellbeing'
c2 = 'workplace'
c3 = c1
topics = getPostInfoForTopicForTwoCategories(c1,c2,c3)
topics.keys()
#topics['Work Life Balance'][0:5]

[u'Family & Friends',
 u'Health & Safety',
 u'Work Life Balance',
 u'Entertainment']

In [45]:
topics['Family & Friends'][0:5]

[{'id': 2426973L,
  'text': u'[corona wk]\nNot at all, but i bought an electric scooter so i dont have to take public transport.\n\nAlso instead of "bless you" we now say "corona".'},
 {'id': 2426973L,
  'text': u'[corona wk]\nNot at all, but i bought an electric scooter so i dont have to take public transport.\n\nAlso instead of "bless you" we now say "corona".'},
 {'id': 2439099L,
  'text': u'Day 5 in quarantine: I miss my fuckin pig ignorant arrogant toxic motherfucker boss and coworkers ! I learned to deal with those motherfucker everyday for 7 Years. \n\nPs: 10 years ago I loved my wife. \U0001f625'},
 {'id': 2439099L,
  'text': u'Day 5 in quarantine: I miss my fuckin pig ignorant arrogant toxic motherfucker boss and coworkers ! I learned to deal with those motherfucker everyday for 7 Years. \n\nPs: 10 years ago I loved my wife. \U0001f625'},
 {'id': 2432760L,
  'text': u"Did some math about corona development in germany...\neven if we immediately lockdown, the health system will 

In [52]:
c1 = 'emergency'
c2 = 'financial'
c3 = c1
topics = getPostInfoForTopicForTwoCategories(c1,c2,c3)
topics.keys()

[u'Coping', u'Awareness']

In [53]:
topics['Coping'][0:10]

[{'id': 2445298L,
  'text': u"I haven't coded for a week which is the longest I've gone without a commit in about 4 years. People are bitching worldwide about deaths and total economy collapse and I'd probably give a shit if forced isolation hadn't completely fucked up a routine that took years for me to perfect.\n\nMy productivity before COVID-19 was showing me six figures and I had a perfect work:life balance. Now my coworkers are just ranting back-and-forth, sharing memes, hypothesising about how much this is going to fuck the planet and what it will mean for our jobs when we finally go back. A few of my coworkers have returned but my department is changing projects so I've got to wait at least another week.\n\nIf it wasn't for my partner starting her first day at a kickass new job tomorrow, my mates and the hobby projects I've got, I don't even know...\n\nI'm glad my country is taking isolation so seriously and I don't care if I'm not making money for a while. I just wish I could h